In [ ]:
import os
import glob
import collections
from pathlib import Path

from astropy.io import ascii
from vega import VegaInterface
import emcee
import corner
import yaml
%pylab inline

import constants
import plotting

In [ ]:
LABEL_MAPPING = {
    'bias_QSO': r'$b_{gal}$', 
    'beta_QSO': r'$\beta_{gal}$', 
    'sigma_velo_disp_gauss_QSO': r'$\sigma_z$',
    'drp_QSO': r'$\delta_z$',
    'bias_hcd': r'$b_{DLA}$',
    'beta_hcd': r'$\beta_{DLA}$',
    'L0_hcd': r'$L_{DLA}$'
}

SURVEY_TITLE_MAPPING = {
    '3dhst': '3D-HST',
    'clamato': 'CLAMATO',
    'mosdef': 'MOSDEF',
    'vuds': 'VUDS',
    'zDeep': 'COSMOS-zDEEP'
}

SUPTITLE_FONT_SIZE = 13

FIG_PNG_DPI = 200

def plot_preamble():
    #plt.rc('font', family='serif')
    plt.rc('xtick', labelsize='medium')
    plt.rc('ytick', labelsize='medium')
    # plt.rc('legend', fontsize=7.5)
    plt.rc('legend', fontsize=10)
    plt.rc('axes', titlesize='x-large')     # fontsize of the axes title
    plt.rc('axes', labelsize='large')    # fontsize of the x and y labels
    
plot_preamble()

In [ ]:
def plot_model_w_data(survey_name, param_dict, model_num_params,
                      SigMin = 0., SigMax = 30.,
                      PiMin = -30., PiMax = 30.):

    PiBins0 = ascii.read(Path(constants.XCORR_DIR_BASE) / 'bins23_pi_0-30hMpc.txt')
    SigBins0 = ascii.read(Path(constants.XCORR_DIR_BASE) / 'bins10_sigma_0-30hMpc.txt')
    cosmo = constants.COSMOLOGY
    
    XCorr_obs = np.load(Path(constants.XCORR_DIR_BASE) / 'obs' / f'xcorr_{survey_name}_globalf_{constants.DATA_VERSION}.npy')

    PiEdges = PiBins0['pi_edges'].data
    SigEdges = SigBins0['sigma_edges'].data

    SigEdgesVec, PiEdgesVec = np.meshgrid(SigEdges, PiEdges)

    SigEdgesPlot = SigEdgesVec / (np.ones(np.shape(SigEdgesVec))*[cosmo.h])
    PiEdgesPlot = PiEdgesVec / (np.ones(np.shape(PiEdgesVec))*[cosmo.h])

    vega = VegaInterface(Path(constants.MCMC_DIR_BASE) / survey_name / f'main_{survey_name}.ini')
    for k, v in param_dict.items():
        vega.params[k] = v
    if 'beta_QSO' not in param_dict:
        vega.params['beta_QSO'] = vega.params['growth_rate'] / vega.params['bias_QSO']
    XModel = vega.compute_model()['qsoxlya'].reshape(*XCorr_obs.shape)
    XModelPlot = XModel.T
        
    # Verify that the model is now in the same shape as the observed (which should be in the 
    # 'original' shape)
    XCorrPlot = np.transpose(XCorr_obs)
    if not np.array_equal(np.shape(XModelPlot),np.shape(XCorrPlot)):
        if np.array_equal(np.shape(np.transpose(XCorrPlot)),np.shape(XModelPlot)):
            XCorrPlot = np.transpose(XCorrPlot)
        else:
            print('Input XCorr array not compatible with model array!')
    
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(5,5))
                            
    ax1.pcolormesh(SigEdgesPlot, PiEdgesPlot, XCorrPlot,cmap='jet_r',vmin=-0.2, vmax=0.1 )
    ax1.set_aspect('equal')
    ax1.set_xlim(0., SigMax)
    ax1.set_ylim(PiMin, PiMax)
    ax1.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
    ax1.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
    # ax1.set_title(survey_name,fontsize=10)
    
    ax2.pcolormesh(SigEdgesPlot, PiEdgesPlot, XModelPlot,cmap='jet_r',vmin=-0.2, vmax=0.1 )
    ax2.set_aspect('equal')
    ax2.set_xlim(SigMin, SigMax)
    ax2.set_ylim(PiMin, PiMax)
    ax2.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
    # ax2.set_title(f'chi-sq = {vega.chi2():.1f}',fontsize=10)
    # print(vega.corr_items['qsoxlya'].rp_rt_grid[1][~vega.data['qsoxlya'].mask])
    fig.suptitle(f'$\chi_r^2$ = {vega.chi2() / (np.sum(vega.data["qsoxlya"].mask) - model_num_params):.3f}', y=0.9, fontsize=SUPTITLE_FONT_SIZE)

In [ ]:
def corner_plot(survey_name, chain_suffix='', show_grid_fit=False, **kwargs):
    backend = emcee.backends.HDFBackend(os.path.join(constants.MCMC_DIR_BASE, survey_name, f'chain_{survey_name}{chain_suffix}.hdf5'), read_only=True)
    config = None
    for p in glob.glob('mcmc_cfg/*.yaml'):
        if p.split('/')[-1].casefold() == f'{survey_name.casefold()}{chain_suffix}.yaml':
            with open(p, 'r') as f:
                config = yaml.safe_load(f)
    if not config:
        raise RuntimeError
    if 'beta_QSO' in config['initial'] and config['initial']['beta_QSO'] is None:
        del config['initial']['beta_QSO']
    tau = backend.get_autocorr_time(quiet=True)
    burnin = int(2 * np.max(tau))
    thin = int(0.5 * np.min(tau))
    # thin = int(np.max(tau))
    chain_len = len(backend.get_chain())
    samples = backend.get_chain(discard=burnin, flat=True, thin=thin)
    
    print(f"Autocorrelation time: {tau}")
    print(f"Chain length: {chain_len}; Chain length / 50: {chain_len / 50:.2f}")
    print("Burn-in: {0}".format(burnin))
    print("Thin: {0}".format(thin))
    print("Flat chain shape: {0}; {1:.1f} autocorrelation times.".format(samples.shape, samples.shape[0] / np.max(tau)))
    
    final_medians = np.median(samples, axis=0)
    unflat_samples = backend.get_chain(discard=burnin, flat=False, thin=thin)
    # print(unflat_samples.shape)
    for i in range(1, unflat_samples.shape[0]):
        subsample = unflat_samples[:i].reshape(-1, samples.shape[-1])
        if np.allclose(np.median(subsample, axis=0), final_medians, rtol=0.05):
            print(f'# chain iterations to arrive at ~5% parameter estimate: {i * thin + burnin}')
            break
    
    if show_grid_fit:
        truths = config['initial'].values()
    else:
        truths = None
        
    # For corner plot, multiply delta_z by -1 so we're consistent with definitions.
    delta_z_inverted_samples = np.copy(samples)
    try:
        delta_z_ind = list(config['initial'].keys()).index('drp_QSO')
        delta_z_inverted_samples[:, delta_z_ind] *= -1
        if truths:
            truths[delta_z_ind] *= -1
    except ValueError:
        pass
    
    # Divide dispersion by 2 to get to what we consider z-dispersion.
    try:
        disp_corrected_samples = np.copy(delta_z_inverted_samples)
        sigma_z_ind = list(config['initial'].keys()).index('sigma_velo_disp_gauss_QSO')
        disp_corrected_samples[:, sigma_z_ind] /= 2
        if truths:
            truths[sigma_z_ind] /= 2
    except ValueError:
        pass
    
    corner.corner(disp_corrected_samples, labels=[LABEL_MAPPING[k] for k in config['initial']], show_titles=True, 
                  truths=truths, truth_color='red',
                  **kwargs)
    plt.suptitle(SURVEY_TITLE_MAPPING[survey_name], x=0.54, weight='bold', fontsize=SUPTITLE_FONT_SIZE)
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'titledcorner_{survey_name}{chain_suffix}.png'), dpi=FIG_PNG_DPI)
    plt.suptitle('')
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'corner_{survey_name}{chain_suffix}.pdf'))
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'corner_{survey_name}{chain_suffix}.png'), dpi=FIG_PNG_DPI)
    plt.show()
    median_params = {}
    for i, k in enumerate(config['initial']):
        median_params[k] = np.median(samples[:, i])
    if 'fixed' in config and config['fixed']:
        for k, p in config['fixed'].items():
            median_params[k] = p
    plot_model_w_data(survey_name, median_params, samples.shape[1])
    plt.tight_layout()
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'bestmodel_{survey_name}{chain_suffix}.pdf'))
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'bestmodel_{survey_name}{chain_suffix}.png'), dpi=FIG_PNG_DPI)
    plt.suptitle(f'{SURVEY_TITLE_MAPPING[survey_name]} | ' + plt.gcf()._suptitle.get_text(), y=0.9, fontsize=SUPTITLE_FONT_SIZE)
    # plt.axhline(-median_params['drp_QSO'] / 0.8, color='black')
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'titledbestmodel_{survey_name}{chain_suffix}.png'), dpi=FIG_PNG_DPI)
    return samples

In [ ]:
corner_plot('vuds');

In [ ]:
corner_plot('zDeep')#, chain_suffix='_tcut');

In [ ]:
mosdef_samples = corner_plot('mosdef');
# plt.show()
# mosdef_params = collections.OrderedDict(
#     bias_QSO=np.median(mosdef_samples[:, 0]),
#     beta_QSO=0.970386193694752 / np.median(mosdef_samples[:, 0]),
#     par_sigma_smooth=np.median(mosdef_samples[:, 1]),
#     drp_QSO=np.median(mosdef_samples[:, 2]),
#     bias_hcd=np.median(mosdef_samples[:, 3]),
#     beta_hcd=np.median(mosdef_samples[:, 4])
# )
# plot_model_w_data('mosdef', mosdef_params)

In [ ]:
corner_plot('clamato')#, chain_suffix='_oned');

In [ ]:
corner_plot('3dhst');
# plt.show()
# hst_params = collections.OrderedDict(
#     bias_QSO=1,
#     beta_QSO=1**-1,
#     sigma_velo_disp_gauss_QSO=0,
#     drp_QSO=0.5,
#     bias_hcd=0,
#     beta_hcd=0
# )
# plot_model_w_data('3dhst', hst_params)

In [ ]:
# corner_plot('all');

In [ ]:
plt.figure(figsize=(1.5, 5))
img = plt.imshow(np.array([[-0.2, 0.1]]), cmap='jet_r')
img.set_visible(False)
plt.axis('off')
plt.colorbar(fraction=1, label=r'$\hat{\xi}_A$')
plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', 'bestmodel_colorbar.png'), dpi=FIG_PNG_DPI)

In [ ]:
pwd = os.getcwd()
os.chdir(os.path.join(constants.FIG_DIR_BASE, 'mcmc'))
os.system('./create-mosaic.sh')
os.chdir(pwd)